In [1]:
import argparse

import numpy as np

import genesis as gs


########################## init ##########################
gs.init(backend=gs.gpu)

########################## create a scene ##########################
viewer_options = gs.options.ViewerOptions(
    camera_pos=(3, -1, 1.5),
    camera_lookat=(0.0, 0.0, 0.0),
    camera_fov=30,
    max_FPS=60,
)

scene = gs.Scene(
    viewer_options=viewer_options,
    rigid_options=gs.options.RigidOptions(
        dt=0.01,
    ),
    show_viewer=True,
)

########################## entities ##########################
plane = scene.add_entity(
    gs.morphs.Plane(),
)
bottle = scene.add_entity(
    material=gs.materials.Rigid(rho=300),
    morph=gs.morphs.URDF(
        file="urdf/3763/mobility_vhacd.urdf",
        scale=0.09,
        pos=(0.65, 0.0, 0.036),
        euler=(0, 90, 0),
    ),
    # visualize_contact=True,
)
target_entity = scene.add_entity(
    gs.morphs.Mesh(
        file="meshes/axis.obj",
        scale=0.15,
    ),
    surface=gs.surfaces.Default(color=(1, 0.5, 0.5, 1)),
)
franka = scene.add_entity(
    gs.morphs.MJCF(file="xml/franka_emika_panda/panda.xml"),
)


[I 05/07/25 21:10:23.515 9139] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


[Genesis] [21:10:26] [INFO] ╭───────────────────────────────────────────────╮
[Genesis] [21:10:26] [INFO] │┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈ Genesis ┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈│
[Genesis] [21:10:26] [INFO] ╰───────────────────────────────────────────────╯
[Genesis] [21:10:26] [INFO] Running on [NVIDIA GeForce RTX 3070 Laptop GPU] with backend gs.cuda. Device memory: 7.78 GB.
[Genesis] [21:10:26] [INFO] 🚀 Genesis initialized. 🔖 version: 0.2.1, 🌱 seed: None, 📏 precision: '32', 🐛 debug: False, 🎨 theme: 'dark'.
[Genesis] [21:10:26] [INFO] Scene <48ea381> created.
[Genesis] [21:10:26] [INFO] Adding <gs.RigidEntity>. idx: 0, uid: <bbcd5c1>, morph: <gs.morphs.Plane>, material: <gs.materials.Rigid>.
[Genesis] [21:10:27] [INFO] Adding <gs.RigidEntity>. idx: 1, uid: <5222392>, morph: <gs.morphs.URDF(file='/home/vahid/envs/panda/lib/python3.10/site-packages/genesis/assets/urdf/3763/mobility_vhacd.urdf')>, material: <gs.materials.Rigid>.
[Genesis] [21:10:27] [INFO] Adding <gs.RigidEntity>. idx: 2, uid: <a487843>, morph

In [2]:

########################## build ##########################
scene.build()

motors_dof = np.arange(7)
fingers_dof = np.arange(7, 9)

# Optional: set control gains
franka.set_dofs_kp(
    np.array([4500, 4500, 3500, 3500, 2000, 2000, 2000, 100, 100]),
)
franka.set_dofs_kv(
    np.array([450, 450, 350, 350, 200, 200, 200, 10, 10]),
)
franka.set_dofs_force_range(
    np.array([-87, -87, -87, -87, -12, -12, -12, -100, -100]),
    np.array([87, 87, 87, 87, 12, 12, 12, 100, 100]),
)


[Genesis] [21:10:32] [INFO] Building scene <48ea381>...
[Genesis] [21:10:33] [WARNING] Reference robot position exceeds joint limits.
[Genesis] [21:10:33] [WARNING] Constraint solver time constant was increased to avoid numerical instability (from `0.005` to `0.02`). Decrease simulation timestep to avoid altering the original value.


[W 05/07/25 21:10:34.691 9139] [frontend_ir.cpp:begin_frontend_struct_for_on_external_tensor@1694] ti.loop_config(serialize=True) does not have effect on the struct for. The execution order is not guaranteed.


[Genesis] [21:10:36] [INFO] Compiling simulation kernels...
[Genesis] [21:11:56] [INFO] Building visualizer...
[Genesis] [21:12:01] [INFO] Viewer created. Resolution: 1200×900, max_FPS: 60.


In [9]:
scene.reset()


[Genesis] [21:25:52] [INFO] Resetting Scene <48ea381>.


In [13]:
scene.reset()
end_effector = franka.get_link("hand")

# move to pre-grasp pose
qpos = franka.inverse_kinematics(
    link=end_effector,
    pos=np.array([0.65, 0.0, 0.25]),
    quat=np.array([0, 1, 0, 0]),
    rot_mask = [False, True, False]
)
qpos[-2:] = 0.04
path = franka.plan_path(qpos)
for waypoint in path:
    franka.control_dofs_position(waypoint)
    scene.step()
    
for _ in range(200):
    scene.step()

[Genesis] [21:27:34] [INFO] Resetting Scene <48ea381>.


[Genesis] [21:27:34] [WARNING] `qpos_start` exceeds joint limit. Relaxing joint limit to contain `qpos_start` for planning.
[Genesis] [21:27:34] [INFO] Ingoring collision pairs already active for starting pos.
[Genesis] [21:27:34] [INFO] Path solution found successfully.
[Genesis] [21:27:34] [INFO] Running at 0.85 FPS.
[Genesis] [21:27:34] [INFO] Running at 0.89 FPS.
[Genesis] [21:27:34] [INFO] Running at 0.94 FPS.
[Genesis] [21:27:34] [INFO] Running at 0.98 FPS.
[Genesis] [21:27:35] [INFO] Running at 1.03 FPS.
[Genesis] [21:27:35] [INFO] Running at 1.09 FPS.
[Genesis] [21:27:35] [INFO] Running at 1.14 FPS.
[Genesis] [21:27:35] [INFO] Running at 1.20 FPS.
[Genesis] [21:27:35] [INFO] Running at 1.26 FPS.
[Genesis] [21:27:35] [INFO] Running at 1.33 FPS.
[Genesis] [21:27:35] [INFO] Running at 1.40 FPS.
[Genesis] [21:27:35] [INFO] Running at 1.47 FPS.
[Genesis] [21:27:35] [INFO] Running at 1.55 FPS.
[Genesis] [21:27:35] [INFO] Running at 1.62 FPS.
[Genesis] [21:27:35] [INFO] Running at 1.7

In [25]:
center = np.array([0.62, 0.0, 0.12])
left_finger = franka.get_link("left_finger")
right_finger = franka.get_link("right_finger")

target_quat = np.array([0, 1, 0, 0])

target_pos_left = center
target_pos_right = target_pos_left 

qpos = franka.inverse_kinematics_multilink(
    links=[left_finger, right_finger],
    poss=[target_pos_left, target_pos_right],
    quats=[target_quat, target_quat],
    rot_mask=[False, False, True],  # only restrict direction of z-axis
)
franka.control_dofs_position(qpos[:-2], motors_dof)
franka.control_dofs_force(np.array([-1, -1]), fingers_dof)
for i in range(100):
    scene.step()


[Genesis] [22:22:35] [INFO] Running at 0.14 FPS.
[Genesis] [22:22:35] [INFO] Running at 0.15 FPS.
[Genesis] [22:22:35] [INFO] Running at 0.16 FPS.
[Genesis] [22:22:35] [INFO] Running at 0.16 FPS.
[Genesis] [22:22:35] [INFO] Running at 0.17 FPS.
[Genesis] [22:22:35] [INFO] Running at 0.18 FPS.
[Genesis] [22:22:35] [INFO] Running at 0.19 FPS.
[Genesis] [22:22:35] [INFO] Running at 0.20 FPS.
[Genesis] [22:22:35] [INFO] Running at 0.21 FPS.
[Genesis] [22:22:35] [INFO] Running at 0.22 FPS.
[Genesis] [22:22:35] [INFO] Running at 0.23 FPS.
[Genesis] [22:22:35] [INFO] Running at 0.25 FPS.
[Genesis] [22:22:35] [INFO] Running at 0.26 FPS.
[Genesis] [22:22:35] [INFO] Running at 0.27 FPS.
[Genesis] [22:22:35] [INFO] Running at 0.29 FPS.
[Genesis] [22:22:35] [INFO] Running at 0.30 FPS.
[Genesis] [22:22:35] [INFO] Running at 0.32 FPS.
[Genesis] [22:22:35] [INFO] Running at 0.34 FPS.
[Genesis] [22:22:35] [INFO] Running at 0.35 FPS.
[Genesis] [22:22:35] [INFO] Running at 0.37 FPS.
[Genesis] [22:22:35]

In [14]:

# reach
qpos = franka.inverse_kinematics(
    link=end_effector,
    pos=np.array([0.65, 0.0, 0.142]),
    quat=np.array([0, 1, 0, 0]),
    rot_mask = [False, True, False]
)
franka.control_dofs_position(qpos[:-2], motors_dof)
for i in range(100):
    scene.step()


[Genesis] [21:28:12] [INFO] Running at 0.61 FPS.


[Genesis] [21:28:12] [INFO] Running at 0.64 FPS.
[Genesis] [21:28:12] [INFO] Running at 0.68 FPS.
[Genesis] [21:28:12] [INFO] Running at 0.71 FPS.
[Genesis] [21:28:12] [INFO] Running at 0.75 FPS.
[Genesis] [21:28:12] [INFO] Running at 0.79 FPS.
[Genesis] [21:28:12] [INFO] Running at 0.83 FPS.
[Genesis] [21:28:12] [INFO] Running at 0.87 FPS.
[Genesis] [21:28:12] [INFO] Running at 0.92 FPS.
[Genesis] [21:28:12] [INFO] Running at 0.97 FPS.
[Genesis] [21:28:12] [INFO] Running at 1.02 FPS.
[Genesis] [21:28:12] [INFO] Running at 1.07 FPS.
[Genesis] [21:28:12] [INFO] Running at 1.12 FPS.
[Genesis] [21:28:12] [INFO] Running at 1.18 FPS.
[Genesis] [21:28:12] [INFO] Running at 1.24 FPS.
[Genesis] [21:28:12] [INFO] Running at 1.31 FPS.
[Genesis] [21:28:12] [INFO] Running at 1.37 FPS.
[Genesis] [21:28:12] [INFO] Running at 1.44 FPS.
[Genesis] [21:28:12] [INFO] Running at 1.52 FPS.
[Genesis] [21:28:12] [INFO] Running at 1.60 FPS.
[Genesis] [21:28:12] [INFO] Running at 1.68 FPS.
[Genesis] [21:28:12]

In [6]:

# grasp
franka.control_dofs_position(qpos[:-2], motors_dof)
franka.control_dofs_position(np.array([0, 0]), fingers_dof)  # you can use position control
for i in range(100):
    scene.step()


[Genesis] [21:14:28] [INFO] Running at 0.94 FPS.
[Genesis] [21:14:28] [INFO] Running at 0.99 FPS.
[Genesis] [21:14:28] [INFO] Running at 1.05 FPS.
[Genesis] [21:14:28] [INFO] Running at 1.10 FPS.
[Genesis] [21:14:28] [INFO] Running at 1.16 FPS.
[Genesis] [21:14:28] [INFO] Running at 1.22 FPS.
[Genesis] [21:14:28] [INFO] Running at 1.28 FPS.
[Genesis] [21:14:28] [INFO] Running at 1.34 FPS.
[Genesis] [21:14:28] [INFO] Running at 1.41 FPS.
[Genesis] [21:14:28] [INFO] Running at 1.48 FPS.
[Genesis] [21:14:28] [INFO] Running at 1.56 FPS.
[Genesis] [21:14:28] [INFO] Running at 1.63 FPS.
[Genesis] [21:14:28] [INFO] Running at 1.71 FPS.
[Genesis] [21:14:28] [INFO] Running at 1.79 FPS.
[Genesis] [21:14:28] [INFO] Running at 1.88 FPS.
[Genesis] [21:14:28] [INFO] Running at 1.97 FPS.
[Genesis] [21:14:28] [INFO] Running at 2.06 FPS.
[Genesis] [21:14:28] [INFO] Running at 2.16 FPS.
[Genesis] [21:14:28] [INFO] Running at 2.26 FPS.
[Genesis] [21:14:29] [INFO] Running at 2.37 FPS.
[Genesis] [21:14:29]

In [7]:

# lift
qpos = franka.inverse_kinematics(
    link=end_effector,
    pos=np.array([0.65, 0.0, 0.3]),
    quat=np.array([0, 1, 0, 0]),
)
franka.control_dofs_position(qpos[:-2], motors_dof)
franka.control_dofs_force(np.array([-20, -20]), fingers_dof)  # can also use force control
for i in range(1000):
    scene.step()

[Genesis] [21:15:03] [INFO] Running at 0.64 FPS.
[Genesis] [21:15:03] [INFO] Running at 0.68 FPS.
[Genesis] [21:15:03] [INFO] Running at 0.71 FPS.
[Genesis] [21:15:03] [INFO] Running at 0.75 FPS.
[Genesis] [21:15:03] [INFO] Running at 0.79 FPS.
[Genesis] [21:15:03] [INFO] Running at 0.82 FPS.
[Genesis] [21:15:03] [INFO] Running at 0.87 FPS.
[Genesis] [21:15:03] [INFO] Running at 0.91 FPS.
[Genesis] [21:15:03] [INFO] Running at 0.96 FPS.
[Genesis] [21:15:03] [INFO] Running at 1.00 FPS.
[Genesis] [21:15:03] [INFO] Running at 1.05 FPS.
[Genesis] [21:15:03] [INFO] Running at 1.11 FPS.
[Genesis] [21:15:03] [INFO] Running at 1.16 FPS.
[Genesis] [21:15:03] [INFO] Running at 1.22 FPS.
[Genesis] [21:15:03] [INFO] Running at 1.28 FPS.
[Genesis] [21:15:03] [INFO] Running at 1.34 FPS.
[Genesis] [21:15:03] [INFO] Running at 1.41 FPS.
[Genesis] [21:15:03] [INFO] Running at 1.48 FPS.
[Genesis] [21:15:03] [INFO] Running at 1.55 FPS.
[Genesis] [21:15:03] [INFO] Running at 1.63 FPS.
[Genesis] [21:15:03]

In [8]:
franka.control_dofs_force(np.array([-0, -0]), fingers_dof)  # can also use force control
for i in range(300):
    scene.step()

[Genesis] [21:16:22] [INFO] Running at 0.63 FPS.
[Genesis] [21:16:22] [INFO] Running at 0.67 FPS.
[Genesis] [21:16:22] [INFO] Running at 0.70 FPS.
[Genesis] [21:16:22] [INFO] Running at 0.74 FPS.
[Genesis] [21:16:22] [INFO] Running at 0.78 FPS.
[Genesis] [21:16:22] [INFO] Running at 0.82 FPS.
[Genesis] [21:16:22] [INFO] Running at 0.86 FPS.
[Genesis] [21:16:22] [INFO] Running at 0.90 FPS.
[Genesis] [21:16:22] [INFO] Running at 0.95 FPS.
[Genesis] [21:16:22] [INFO] Running at 1.00 FPS.
[Genesis] [21:16:22] [INFO] Running at 1.05 FPS.
[Genesis] [21:16:22] [INFO] Running at 1.10 FPS.
[Genesis] [21:16:22] [INFO] Running at 1.16 FPS.
[Genesis] [21:16:22] [INFO] Running at 1.22 FPS.
[Genesis] [21:16:22] [INFO] Running at 1.28 FPS.
[Genesis] [21:16:22] [INFO] Running at 1.35 FPS.
[Genesis] [21:16:22] [INFO] Running at 1.42 FPS.
[Genesis] [21:16:22] [INFO] Running at 1.49 FPS.
[Genesis] [21:16:22] [INFO] Running at 1.57 FPS.
[Genesis] [21:16:22] [INFO] Running at 1.65 FPS.
[Genesis] [21:16:22]